In [11]:
!pip install -q groq langchain_community langchain_groq

In [12]:
import os
from groq import Groq
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_groq import ChatGroq

In [13]:
GROQ_API_KEY= "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"  # Replace with your actual API key

In [14]:
llm = ChatGroq(
    api_key =GROQ_API_KEY,
    model= "llama3-8b-8192",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [15]:
custom_prompt_template_for_chatbot = """
You are a knowledgeable assistant specializing in Data Science and Artificial Intelligence (AI).

Your primary objective is to assist students by providing clear, concise, and accurate answers to their questions specifically related to Data Science and AI. This includes, but is not limited to, the following topics:
- Programming languages and tools: Python, SQL (MySQL, SQLite, MongoDB)
- Data visualization tools: Power BI, Tableau
- Statistical concepts and methodologies
- Machine Learning (ML) techniques and frameworks
- MLFlow for managing machine learning workflows
- Containerization with Docker
- Deep Learning concepts and frameworks
- Natural Language Processing (NLP)
- Generative AI technologies
- Skills required for a career in Data Science and AI

When responding, ensure that your answers are focused and straightforward, avoiding unnecessary details. If users ask complex questions, break down your responses into manageable parts and provide step-by-step explanations when needed.

Always be polite and encouraging, ensuring that you provide accurate information at all times.

If a question is asked that falls outside the realm of Data Science and AI or does not relate to the topics mentioned above, respond with a polite message indicating that the question is unrelated. For example: "I'm sorry, but that topic is outside the scope of Data Science and AI. I'm unable to provide an answer."

Remember previous exchanges in the conversation to provide better context for your responses.

History: {history}

Context: {context}

Question: {question}
"""


In [16]:
prompt = PromptTemplate(template=custom_prompt_template_for_chatbot,
                            input_variables=['history','context', 'question'])

memory = ConversationBufferMemory(input_key="question", memory_key="history")

In [17]:
# Function to format output in Markdown
def format_output(response):
    if isinstance(response, str):
        # Check if the response contains code (you can customize this check)
        if response.startswith("``````"):
            return response  # Return as is if it's already in code block
        else:
            # Format as markdown for theoretical responses
            formatted_response = f"# Response\n\n{response}"
            return formatted_response
    return response

In [18]:
chain = prompt | llm

def invoke_chain(question):
    response = chain.invoke({"history": memory, "context": "", "question": question})
    return format_output(response)

In [19]:
question = "write a python code to train linear regression model from taking data till predictions"
response = invoke_chain(question)
print(response)

content="I'd be happy to help you with that!\n\nTo train a linear regression model in Python, we'll use the scikit-learn library. Here's a step-by-step code to get you started:\n```python\nimport pandas as pd\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.linear_model import LinearRegression\nfrom sklearn.metrics import mean_squared_error\n\n# Load your dataset (replace with your own data)\ndf = pd.read_csv('your_data.csv')\n\n# Split the data into features (X) and target (y)\nX = df.drop('target_column', axis=1)  # replace with your target column name\ny = df['target_column']  # replace with your target column name\n\n# Split the data into training and testing sets\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# Create a linear regression model\nmodel = LinearRegression()\n\n# Train the model on the training data\nmodel.fit(X_train, y_train)\n\n# Make predictions on the testing data\ny_pred = model.predict(X_test)\n\

In [20]:
while True:
    question = input("Ask a question (or type 'exit' to quit): ")
    if question.lower() == 'exit':
        break

    response = invoke_chain(question)
    print(response)